# Обслуживание клиентов банка

Рассмотрим процесс обслуживания клиентов банка в одном из его офисов.
Пусть есть $m$ сотрудников, готовых обслуживать клиентов, и $n$ терминалов самообслуживания.

Каждый $j$-ый сотрудник, $j=1,2,\ldots,m$, имеет собственное заданное время работы: от $t_{\text{s} j}$ до $t_{\text{e} j}$.
При этом, если работник обслуживает клиента, то завершает работу он только после того, как обслужит этого клиента.
Терминалы работают постоянно.

Клиенты приходят в банк в случайные моменты времени с момента открытия банка и вплоть до его закрытия. Пусть появление клиентов подчиняется экспоненциальному распределению:

$$
t_\text{client} \sim \frac{1}{\tau} \exp\left( -\frac{t}{\tau} \right),
$$

где $\tau$ — среднее время, через которое приходит новый клиент.

Допустим, что каждый $i$-ый клиент имеет определённое терпение: он готов ждать некоторое время $t_{\text{w} i}$.
Пусть это будет равномерно распределённая случайная величина $U[t_\text{w min}; t_\text{w max}]$, где положительные $t_\text{w min}$ и $t_\text{w max}$ заданы.
Если клиент в течение времени $t_{\text{w} i}$ не обслуживается, он уходит, разочаровывается в банке и отказывается от его услуг.
Клиент обслуживается в течение времени $t_{\text{sc} i} \sim U[t_\text{sc min}; t_\text{sc max}]$, где пределы $t_\text{sc min}$ и $t_\text{sc max}$ равномерного распределения также заданы и положительны.

**Требуется** промоделировать описанную систему в течении заданного времени $T$.
В качестве результата может быть интересно получить статистику потерянных клиентов, среднее время ожидания клиентов и его распределение и т.п.
Модель, которую мы сформируем можно использовать для решения разнообразных оптимизационных задач.
Например, оптимизировать расписание работы сотрудников, количество терминалов и работников и т.д.

Компьютерную модель создадим с использованием библиотеки для дискретно-событийного моделирования [SimPy](https://simpy.readthedocs.io/en/latest/).

## Компьютерная модель

Подключим библиотеки и функции, которые будем использовать:

In [1]:
import simpy as sim
from random import expovariate

### Клиент

### Процесс, "порождающий" клиентов